In [5]:
import numpy as np
from scipy import linalg

In [2]:
dat = np.random.normal(0,1,size=(100,50))

array([[ 1.8382314 ,  0.70203458, -1.39907614, ..., -0.41432479,
        -0.65410186, -3.1628597 ],
       [-0.72541306, -0.11351731,  2.18984176, ..., -1.46168456,
         0.80348212, -0.44685412],
       [ 0.81617782, -0.60375056,  0.08047425, ..., -0.26380074,
         1.76627808,  0.65802094],
       ..., 
       [-0.47500271,  1.22967018,  1.83431448, ...,  0.27520625,
        -0.97348561,  1.29683127],
       [-2.6869616 ,  0.08472115,  0.79288446, ...,  1.03290265,
        -0.06353873, -0.55631308],
       [ 0.97417549,  0.68859456, -1.19357265, ...,  0.81405408,
         0.02360276, -0.71805652]])

In [11]:
U, s, Vh = linalg.svd(dat, full_matrices=False)
print U.shape, Vh.shape, s.shape
S = linalg.diagsvd(s, 50, 50)
np.allclose(dat, np.dot(U, np.dot(S, Vh)))

(100, 50) (50, 50) (50,)


True

array([[ 0.02920104, -0.03469249, -0.27310824, ..., -0.0698831 ,
        -0.08954962,  0.08039656],
       [-0.04275601, -0.00496401,  0.14076205, ..., -0.07565306,
        -0.03551305,  0.00621941],
       [ 0.06698798, -0.16411406, -0.03379661, ...,  0.12456229,
        -0.03681171,  0.1070639 ],
       ..., 
       [ 0.02072126,  0.06144194, -0.0792387 , ...,  0.01097556,
         0.01310064, -0.03660678],
       [-0.05111623, -0.01551426,  0.11488279, ...,  0.04319157,
         0.08968629,  0.04635527],
       [-0.02196653, -0.04209508,  0.05314878, ...,  0.04101886,
         0.10010793,  0.09959855]])